# Calculate SUV from DICOM
### JA Engelbrecht

In [1]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import sys
import os
from pydicom import dcmread
from pydicom.data import get_testdata_file

def StrToSec(string):
    hours = int(string[0:2])
    minutes = int(string[2:4])
    seconds = float(string[4:12])
    totalSec = hours*3600 + minutes*60 + seconds
    return totalSec

#### Initialise Readers

In [2]:
reader = sitk.ImageSeriesReader()
file_reader = sitk.ImageFileReader()

#### Scan Path

In [3]:
path = 'E:\\All Biomarker Dicoms_jamdre\\S13\\1\\3_PET WB'

In [4]:
dicom_names = reader.GetGDCMSeriesIDs(path)
print(dicom_names)

('1.3.12.2.1107.5.1.4.48568.30000010110506254140600000094', '1.3.12.2.1107.5.1.4.48568.30000011031805533554600000094')


In [5]:
files = reader.GetGDCMSeriesFileNames(path,dicom_names[0])
files = files[::-1]# Resort from big to small
ds = dcmread(files[0])

In [6]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 202
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Positron Emission Tomography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.48568.30000010110506254140600000095
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.5.4
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_354'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0016) SOP Class UID                       UI: Positron Emission Tomography Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.48568.300000101105062541406000

In [7]:
for s in dicom_names:
    files = reader.GetGDCMSeriesFileNames(path,s)
    files = files[::-1]# Resort from big to small
    ds = dcmread(files[0])
    file_reader.SetFileName(files[0])
    file_reader.ReadImageInformation()
    SeriesDescription = file_reader.GetMetaData('0008|1030')
    print(SeriesDescription)
    RescaleIntercept = float(ds.RescaleIntercept)
    RescaleSlope = float(ds.RescaleSlope)
    DoseCalibrationFactor = float(ds.DoseCalibrationFactor)
    PatientWeight = float(ds.PatientWeight)
    RadionuclideTotalDose = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
    RadionuclideHalfLife = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
    SeriesTime = StrToSec(ds.SeriesTime)
    RadiopharmaceuticalStartTime = StrToSec(ds.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime)
    ActualActivity = RadionuclideTotalDose *np.exp(-(np.log(2)*(SeriesTime-RadiopharmaceuticalStartTime))/(RadionuclideHalfLife))*1e-3
    #SUV_Factor = ((RescaleSlope + RescaleIntercept)*DoseCalibrationFactor)/(ActualActivity/PatientWeight)
    SUV_Factor = ((1)/(ActualActivity/PatientWeight))
    reader.SetFileNames(files)
    DicomImage = reader.Execute()
    DicomArray = sitk.GetArrayFromImage(DicomImage)
    SUV = DicomArray* SUV_Factor
    print("Max SUV:",np.max(SUV),"\t Max Array",DicomArray.max())

PET^PETCT_AC_WB_CPT (Adult) 
Max SUV: 10.338522496153988 	 Max Array 29343.89353060725
PET^PETCT_AC_WB_CPT (Adult) 
Max SUV: 10.547840562414093 	 Max Array 27196.836029291055


In [11]:
SUV.shape

(132, 168, 168)

### File by File Analyses

In [ ]:
file_reader.Execute()
files = reader.GetGDCMSeriesFileNames(path,dicom_names[1])
files = files[::-1]# Resort from big to small
ds = dcmread(files[0])
SUV_Array = np.zeros((np.size(files),ds.Rows, ds.Columns))
PET_Array = np.zeros((np.size(files),ds.Rows, ds.Columns))
cnt = 0
for file in files:
    ds = dcmread(file)
    RescaleIntercept = float(ds.RescaleIntercept)
    RescaleSlope = float(ds.RescaleSlope)
    DoseCalibrationFactor = float(ds.DoseCalibrationFactor)
    PatientWeight = float(ds.PatientWeight)
    RadionuclideTotalDose = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
    RadionuclideHalfLife = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
    SeriesTime = StrToSec(ds.SeriesTime)
    RadiopharmaceuticalStartTime = StrToSec(ds.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime)
    ActualActivity = RadionuclideTotalDose *np.exp(-(np.log(2)*(SeriesTime-RadiopharmaceuticalStartTime))/(RadionuclideHalfLife))*1e-3
    LargestImagePixelValue = ds.LargestImagePixelValue
    #SUV_Factor = ((RescaleSlope + RescaleIntercept)*DoseCalibrationFactor)/(ActualActivity/PatientWeight)
    SUV_Factor = ((1)/(ActualActivity/PatientWeight))
    file_reader.SetFileName(file)
    DicomImage = file_reader.Execute()
    DicomArray = sitk.GetArrayFromImage(DicomImage)
    
    scale = LargestImagePixelValue/np.max(DicomArray)
    
    SUV = DicomArray* SUV_Factor
    SUV_Array[cnt,:,:] = SUV
    PET_Array[cnt,:,:] = DicomArray#*scale
    print("Max SUV:",np.max(SUV),"\t Max Array",np.max(DicomArray), "\t Largest Pixel",LargestImagePixelValue/np.max(DicomArray))
    cnt=cnt+1
print("Max SUV:",np.max(SUV_Array),"\t Max Array:",np.max(PET_Array))